In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from IPython.display import clear_output

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class P:
    device='cuda:4'
    env='CartPole-v1'
    num_envs=3
    hidden_units=128
    lr=3e-4
    num_steps_td=5
    max_steps=20000
    gamma=0.99
    tau=0.95

# Envs

In [3]:
envs = gym.vector.make(P.env, num_envs=P.num_envs)
test_env = gym.make(P.env)

# Neural Network

In [4]:
class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size):
        super(ActorCritic, self).__init__()

        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        )

        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        value = self.critic(x)
        probs = self.actor(x)
        dist = Categorical(probs)
        return dist, value

In [5]:
num_inputs = envs.single_observation_space.shape[0]
num_outputs = envs.single_action_space.n
num_inputs, num_outputs

(4, 2)

In [6]:
model = ActorCritic(num_inputs, num_outputs, P.hidden_units).to(P.device)
optimizer = optim.Adam(model.parameters(), lr=P.lr)

# Helper funcs

In [7]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.title("frame %s. reward: %s" % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()

In [8]:
def evaluate(env, model, device):
    state, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, termination, truncation, info = env.step(dist.sample().cpu().numpy()[0])
        done = termination or truncation
        state = next_state
        total_reward += reward
    return total_reward

# GAE

In [9]:
def compute_gae(next_value, rewards, masks, values, gamma, tau):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

# A2C

In [ ]:
frame_idx = 0
test_rewards = []

state, _ = envs.reset()
while frame_idx < P.max_steps:
    log_probs = []
    values = []
    rewards = []
    masks = []
    entropy = 0

    for _ in range(P.num_steps_td):
        state = torch.FloatTensor(state)
        state = state.to(P.device)
        dist, value = model(state)

        action = dist.sample()
        next_state, reward, terminations, truncations, infos = envs.step(action.cpu().numpy())
        done = np.logical_or(terminations, truncations)

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()

        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(P.device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(P.device))

        state = next_state
        frame_idx += 1
        if frame_idx % 1000 == 0:
            print(frame_idx)
            test_rewards.append(np.mean([evaluate(test_env, model, P.device) for _ in range(10)]))
            plot(frame_idx, test_rewards)

    # Bootstrap to get returns in n-step
    next_state = torch.FloatTensor(state).to(P.device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values, P.gamma, P.tau)

    log_probs = torch.cat(log_probs)
    returns = torch.cat(returns).detach()
    values = torch.cat(values)

    advantage = returns - values

    # Losses
    actor_loss = -(log_probs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()
    loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

evaluate(test_env, model, P.device)